# CS50 Final Project

## FIFA World Cup Predictions

In [15]:
import pandas as pd
import numpy as np
from scipy.stats import poisson

#Import Dataset "wcmatches_history.csv" and clean the data for analysis
wc_games = pd.read_csv('538_wc_matches.csv', sep=';')
wc_games = wc_games.drop(columns=['league_id', 'league', 'spi1', 'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1', 'adj_score2'])

# Import Dataset with 2022 FIFA World Cup's Data
data = pd.read_excel('DadosCopaDoMundoQatar2022.xlsx', sheet_name='selecoes').rename(columns={'NomeEmIngles': 'team'}).set_index('team').rename(index={'United States': 'USA'})

# Create list of teams
teams = []
[teams.append(team) for team in data.index if team not in teams]

# Import Dataset "international_matches.csv" and clean the data for analysis
hist_games = pd.read_csv('international_matches_filtered.csv', sep=';')

# Transform matches' dates into years
for i in hist_games.index:
    hist_games.at[i, 'date'] = hist_games['date'][i][-4:]
hist_games = hist_games.rename(columns={'date': 'year'})
hist_games['year'] = hist_games['year'].astype(int)
hist_games = hist_games.sort_values('year', ascending=False)

# Create Dataframe with team's status
columns = ['fifa_rank', 'fifa_points', 'goalkeeper_score', 'defense_score', 'offense_score', 'midfield_score']
team_stats = pd.DataFrame(index= teams, columns= columns)

for team in team_stats.index:
    for i in hist_games.index:
        if team == hist_games['home_team'][i]:
            team_stats['fifa_rank'][team] = hist_games['home_team_fifa_rank'][i]
            team_stats['fifa_points'][team] = hist_games['home_team_total_fifa_points'][i]
            team_stats['goalkeeper_score'][team] = hist_games['home_team_goalkeeper_score'][i]
            team_stats['defense_score'][team] = hist_games['home_team_mean_defense_score'][i]
            team_stats['offense_score'][team] = hist_games['home_team_mean_offense_score'][i]
            team_stats['midfield_score'][team] = hist_games['home_team_mean_midfield_score'][i]
            break
        elif team == hist_games['away_team'][i]:
            team_stats['fifa_rank'][team] = hist_games['away_team_fifa_rank'][i]
            team_stats['fifa_points'][team] = hist_games['away_team_total_fifa_points'][i]
            team_stats['goalkeeper_score'][team] = hist_games['away_team_goalkeeper_score'][i]
            team_stats['defense_score'][team] = hist_games['away_team_mean_defense_score'][i]
            team_stats['offense_score'][team] = hist_games['away_team_mean_offense_score'][i]
            team_stats['midfield_score'][team] = hist_games['away_team_mean_midfield_score'][i]
            break

team_stats['fifa_rank'] = team_stats['fifa_rank'].astype(int)
team_stats['fifa_points'] = team_stats['fifa_points'].astype(int)
team_stats = team_stats.sort_values('fifa_points', ascending= False)

team_stats.at['Qatar', 'goalkeeper_score'] = round((team_stats['goalkeeper_score']['Saudi Arabia'] * team_stats['fifa_points']['Qatar']) / team_stats['fifa_points']['Saudi Arabia'], 1)
team_stats.loc['Qatar', 'defense_score'] = round((team_stats['defense_score']['Saudi Arabia'] * team_stats['fifa_points']['Qatar']) / team_stats['fifa_points']['Saudi Arabia'], 1)
team_stats.loc['Qatar', 'offense_score'] = round((team_stats['offense_score']['Saudi Arabia'] * team_stats['fifa_points']['Qatar']) / team_stats['fifa_points']['Saudi Arabia'], 1)
team_stats.loc['Qatar', 'midfield_score'] = round((team_stats['midfield_score']['Saudi Arabia'] * team_stats['fifa_points']['Qatar']) / team_stats['fifa_points']['Saudi Arabia'], 1)
team_stats.loc['Tunisia', 'goalkeeper_score'] = round((team_stats['goalkeeper_score']['Canada'] * team_stats['fifa_points']['Tunisia']) / team_stats['fifa_points']['Canada'], 1)

# Find average goals per match in previous FIFA World Cups
tournaments = hist_games.groupby('tournament').mean(numeric_only= True)
mgoals = (tournaments['home_team_score']['FIFA World Cup'] + tournaments['away_team_score']['FIFA World Cup'])

# Remove matches prior to 2018 from history, to use only matches from the last 5 years
for i in range(len(hist_games['year'])):
    if hist_games['year'][i] < 2018:
        hist_games = hist_games.drop(i)
hist_games = hist_games.reset_index(drop=True)

#Remove Friendly games from dataset
for i in hist_games.index:
    if hist_games['tournament'][i] == 'Friendly':
        hist_games = hist_games.drop(index= i)
hist_games = hist_games.reset_index(drop=True)

In [16]:
display(hist_games)
hist_games.info()

,year,home_team,away_team,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,tournament,country,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,2022,Netherlands,Wales,10,18,1658,1588,3,2,UEFA Nations League,Netherlands,Win,81.0,74.0,85.2,83.0,83.5,75.0,73.0,78.5
1,2022,Poland,Wales,26,18,1544,1588,2,1,UEFA Nations League,Poland,Win,87.0,74.0,75.2,84.7,75.8,75.0,73.0,78.5
2,2022,Costa Rica,USA,42,13,1464,1643,2,0,FIFA World Cup qualification,Costa Rica,Win,88.0,77.0,71.5,70.3,69.2,75.8,77.7,75.8
3,2022,Saudi Arabia,Australia,53,37,1433,1486,1,0,FIFA World Cup qualification,Saudi Arabia,Win,70.0,77.0,72.8,67.7,72.8,72.0,72.3,73.5
4,2022,Ecuador,Argentina,44,4,1458,1766,1,1,FIFA World Cup qualification,Ecuador,Draw,71.0,84.0,73.5,76.0,74.5,82.2,89.0,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2018,Poland,Portugal,18,7,1537,1606,2,3,UEFA Nations League,Poland,Lose,84.0,84.0,78.0,82.0,77.2,81.8,85.0,83.8
194,2018,Spain,Croatia,9,4,1580,1643,6,0,UEFA Nations League,Spain,Win,91.0,83.0,87.8,86.3,87.5,81.0,81.7,86.8
195,2018,Japan,Costa Rica,55,32,1392,1471,3,0,Kirin Challenge Cup,Japan,Win,72.0,87.0,76.0,76.3,79.5,74.5,72.0,71.0
196,2018,France,Netherlands,1,17,1726,1540,2,1,UEFA Nations League,France,Win,88.0,81.0,84.8,83.7,88.2,82.0,83.0,82.2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   year                           198 non-null    int64  
 1   home_team                      198 non-null    object 
 2   away_team                      198 non-null    object 
 3   home_team_fifa_rank            198 non-null    int64  
 4   away_team_fifa_rank            198 non-null    int64  
 5   home_team_total_fifa_points    198 non-null    int64  
 6   away_team_total_fifa_points    198 non-null    int64  
 7   home_team_score                198 non-null    int64  
 8   away_team_score                198 non-null    int64  
 9   tournament                     198 non-null    object 
 10  country                        198 non-null    object 
 11  home_team_result               198 non-null    object 
 12  home_team_goalkeeper_score     196 non-null    flo

In [17]:
team_stats

,fifa_rank,fifa_points,goalkeeper_score,defense_score,offense_score,midfield_score
Brazil,1,1832,89.0,84.8,86.3,85.5
Belgium,2,1827,89.0,80.8,85.7,85.5
France,3,1789,87.0,84.2,88.3,86.8
Argentina,4,1766,84.0,82.2,89.0,84.0
England,5,1755,83.0,85.0,88.0,84.0
Spain,7,1709,84.0,86.5,85.0,86.0
Portugal,8,1674,82.0,85.2,86.0,84.5
Netherlands,10,1658,81.0,85.2,83.0,83.5
Denmark,9,1654,85.0,80.2,77.7,80.2
Germany,11,1648,90.0,84.0,82.7,87.8


In [18]:
data

,Seleção,Grupo,GrupoNumero,PosiçãoRankingFIFA,ValorDeMercado,PontosRankingFIFA,Confederação,Copas,JogadorDestaque,FotoJogadorDestaque,LinkBandeiraPequena,LinkBandeiraGrande
team,,,,,,,,,,,,
Qatar,Catar,A,A1,48,18.33,1442,AFC,0,Almoez Ali,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Ecuador,Equador,A,A2,44,125.80,1464,UEFA,0,Pervis Estupiñán,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Senegal,Senegal,A,A3,18,286.50,1585,CAF,0,Sadio Mané,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Netherlands,Holanda,A,A4,8,455.75,1679,UEFA,0,Virgil Van Dijk,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
England,Inglaterra,B,B1,5,1360.00,1737,UEFA,1,Harry Kane,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Iran,Irã,B,B2,22,68.58,1559,AFC,0,Sardar Azmoun,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
USA,Estados Unidos,B,B3,14,204.40,1635,CONCACAF,0,Christian Pulisic,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Wales,País de Gales,B,B4,19,130.85,1582,UEFA,0,Gareth Bale,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Argentina,Argentina,C,C1,3,608.00,1771,CONMEBOL,2,Lionel Messi,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...


In [19]:
# Define function to identify average of goals in previous matches between 2 teams, if there's none in history return World Cup's avg
def avg_goals(team1, team2):
    if team1 not in teams:
        raise ValueError(f'{team1} not  in the  World Cup!')
    elif team2 not in teams:
        raise ValueError(f'{team2} not  in the  World Cup!')

    goals = 0
    matches = 0

    for i in hist_games.index:
        if (hist_games['home_team'][i] == team1 and hist_games['away_team'][i] == team2) or (hist_games['home_team'][i] == team2 and hist_games['away_team'][i] == team1):
            goals += hist_games['home_team_score'][i] + hist_games['away_team_score'][i]
            matches += 1

    try:
        return goals / matches
    except:
        return mgoals

# Define function to identify the power of each team playing
def lam(team1, team2):
    goals = avg_goals(team1, team2)
    fifa1, fifa2 = team_stats['fifa_points'][team1], team_stats['fifa_points'][team2]
    off1, off2 = team_stats['offense_score'][team1], team_stats['offense_score'][team2]
    def1, def2 = team_stats['defense_score'][team1], team_stats['defense_score'][team2]
    mid1, mid2 = team_stats['midfield_score'][team1], team_stats['midfield_score'][team2]
    gk1, gk2  = team_stats['goalkeeper_score'][team1], team_stats['goalkeeper_score'][team2]
    
    pwr1 = (fifa1 / fifa2) * ((0.9 * off1 + 0.1 * mid1) / (0.8 * def2 + 0.2 * gk2))
    pwr2 = (fifa2 / fifa1) * ((0.9 * off2 + 0.1 * mid2) / (0.8 * def1 + 0.2 * gk1))

    l1 = goals * pwr1 / (pwr1 + pwr2)
    l2 = goals * pwr2 / (pwr1 + pwr2)

    return l1, l2

# Define function to identify the result of each match
def result(goals1, goals2):
    if goals1 > goals2:
        return 'W'
    elif goals2 > goals1:
        return 'L'
    else:
        return 'D'

# Define function to distribute the points based on matches' results
def points(goals1, goals2):
    rst = result(goals1, goals2)
    if rst == 'W':
        pts1, pts2 = 3, 0
    if rst == 'L':
        pts1, pts2 = 0, 3
    if rst == 'D':
        pts1, pts2 = 1, 1
    return pts1, pts2

# Define function to simulate each game
def game(team1, team2):
    l1, l2 = lam(team1, team2)
    goals1 = int(np.random.poisson(lam=l1 , size=1))
    goals2 = int(np.random.poisson(lam=l2 , size=1))
    gd1 = goals1 - goals2
    gd2 = -gd1
    rst = result(goals1, goals2)
    pts1, pts2 = points(goals1, goals2)
    scoreboard = f'{goals1}x{goals2}'
    return {'goals1': goals1, 'goals2': goals2, 'gd1': gd1, 'gd2': gd2, 
            'pts1': pts1, 'pts2': pts2, 'rst': rst, 'scoreboard': scoreboard}

#  Define function to calculate the probabilities of the scoreboard for each game
def distribution(avg):
    probs = []
    for i in range(7):
        probs.append(poisson.pmf(i, avg))
    probs.append(1 - sum(probs))
    return pd.Series(probs, index= ['0', '1', '2', '3', '4', '5', '6', '7+'])

# Define probabilities of each game's result
def match_probs(team1, team2):
    l1, l2 = lam(team1, team2)
    d1, d2 = distribution(l1), distribution(l2)
    matrix = np.outer(d1, d2)
    
    sort = np.sort(matrix)
    max_values = []
    for i in sort:
        for j in i:
            max_values.append(j)
    max_values.sort(reverse= True)
    scoreboards = []
    for i in range(5):
        a = int(np.where(matrix == max_values[i])[0])
        b = int(np.where(matrix == max_values[i])[1])
        scoreboards.append((f'{max_values[i] * 100 :.1f}%', f'{a}x{b}'))


    w = np.tril(matrix).sum() - np.trace(matrix)
    l = np.triu(matrix).sum() - np.trace(matrix)
    d = np.trace(matrix)

    probs = np.around([w, d, l], 3)
    probsp = [f'{100 * i :.1f}%'  for i in  probs]
    
    names = ['0', '1', '2', '3', '4', '5', '6', '7+']
    matrix = pd.DataFrame(matrix, columns = names, index = names)
    matrix.index = pd.MultiIndex.from_product([[team1], matrix.index])
    matrix.columns = pd.MultiIndex.from_product([[team2], matrix.columns])

    return {'probabilities': probsp, 'scoreboards': scoreboards, 'matrix': matrix}


In [20]:
wc_games['group'] = None
for i in wc_games.index:
    wc_games['group'][i] = data['Grupo'][wc_games['team1'][i]]

wc_games['win1'] = None
wc_games['draw'] = None
wc_games['win2'] = None

for i in wc_games.index:
    team1, team2 = wc_games['team1'][i], wc_games['team2'][i]
    w, d, l = match_probs(team1, team2)['probabilities']
    wc_games['win1'][i] = w
    wc_games['draw'][i] = d
    wc_games['win2'][i] = l

wc_games

,date,team1,team2,group,win1,draw,win2
0,20/11/2022,Qatar,Ecuador,A,32.6%,26.3%,41.1%
1,21/11/2022,England,Iran,B,54.4%,24.4%,21.2%
2,21/11/2022,Senegal,Netherlands,A,30.4%,26.1%,43.5%
3,21/11/2022,USA,Wales,B,40.7%,26.3%,33.1%
4,22/11/2022,Argentina,Saudi Arabia,C,61.2%,22.6%,16.2%
5,22/11/2022,Denmark,Tunisia,D,48.9%,25.4%,25.7%
6,22/11/2022,Mexico,Poland,C,40.2%,26.3%,33.5%
7,22/11/2022,France,Australia,D,61.3%,21.0%,17.7%
8,23/11/2022,Morocco,Croatia,F,36.6%,26.4%,37.0%
9,23/11/2022,Germany,Japan,E,47.8%,25.6%,26.6%
